# FXの将来のリターン動向の予測モデル

目的：過去18時間の **"usdjpy","cadjpy","audjpy","eurjpy","zarjpy","sgdjpy"** を使って、将来**1時間後**のリターン動向を予測する   
モデル：GCN   
開発環境: python 3.11.5/ JupyterLab 3.6.3/Jupyter Notebook Version: 6.5.4/System: Linux #14~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch_geometric.nn.pool import global_mean_pool

import numpy as np
import pandas as pd

from tslearn.metrics import dtw
from sklearn.metrics import precision_score, recall_score, roc_auc_score, average_precision_score, f1_score, accuracy_score
from tqdm.auto import tqdm

Pandas version: 2.0.3   
Numpy version: 1.24.3   
sklearn: 1.3.0   
sklearn: 0.6.3   
torch: 2.1.1   
torch_geometric: 2.4.0   
tqdm: 4.65.0

In [2]:
# Load datasets
'''
9 currency pairs from 2018-01-01 18:10:00 to 2023-11-17 16:10:00
'''

def load_dataset(filename, start_date, end_date):
    
    df = pd.read_csv(f'processed_data/{filename}.csv',sep = ',')
    df.set_index('Times', inplace=True)
    
    df = df.loc[start_date:end_date]
    df.reset_index(inplace=True)
    
    return df

start_date = '2022-01-17 18:10:00'
end_date = '2023-11-17 16:10:00'

variable_names = ["usdjpy","cadjpy","audjpy","eurjpy","zarjpy","sgdjpy"]
for name in variable_names:
    exec(f"{name} = load_dataset(str.upper('{name}'), start_date, end_date)")
    exec(f"display({name}.head())")

,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2022-01-17 18:10:00,114.597,0.264026,-0.000044,0,0
1,2022-01-17 18:11:00,114.603,0.264145,0.000026,1,0
2,2022-01-17 18:12:00,114.604,0.264164,0.000009,1,0
3,2022-01-17 18:13:00,114.611,0.264302,0.000044,1,0
4,2022-01-17 18:14:00,114.610,0.264283,0.000026,1,0


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2022-01-17 18:10:00,91.550,0.474199,0.000033,1,1
1,2022-01-17 18:11:00,91.557,0.474387,0.000317,1,1
2,2022-01-17 18:12:00,91.555,0.474334,0.000186,1,1
3,2022-01-17 18:13:00,91.556,0.474360,0.000284,1,1
4,2022-01-17 18:14:00,91.549,0.474173,0.000197,1,1


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2022-01-17 18:10:00,82.608,0.586063,-0.000061,0,2
1,2022-01-17 18:11:00,82.615,0.586244,0.000000,1,2
2,2022-01-17 18:12:00,82.617,0.586296,0.000242,1,2
3,2022-01-17 18:13:00,82.598,0.585804,-0.000182,0,2
4,2022-01-17 18:14:00,82.579,0.585311,-0.000278,0,2


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2022-01-17 18:10:00,130.726,0.326747,-0.000076,0,3
1,2022-01-17 18:11:00,130.739,0.327008,0.000031,1,3
2,2022-01-17 18:12:00,130.738,0.326988,0.000008,1,3
3,2022-01-17 18:13:00,130.729,0.326807,-0.000138,0,3
4,2022-01-17 18:14:00,130.722,0.326667,-0.000084,0,3


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2022-01-17 18:10:00,7.425,0.499865,-0.001442,0,4
1,2022-01-17 18:11:00,7.424,0.499596,-0.001484,0,4
2,2022-01-17 18:12:00,7.423,0.499327,-0.001527,0,4
3,2022-01-17 18:13:00,7.423,0.499327,-0.001435,0,4
4,2022-01-17 18:14:00,7.436,0.502825,0.000407,1,4


,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2022-01-17 18:10:00,84.945,0.301345,-0.000165,0,5
1,2022-01-17 18:11:00,84.953,0.301549,0.000436,1,5
2,2022-01-17 18:12:00,84.954,0.301575,0.000224,1,5
3,2022-01-17 18:13:00,84.959,0.301703,0.000247,1,5
4,2022-01-17 18:14:00,84.956,0.301626,0.000188,1,5


In [3]:
# Combine all the datasets
combined_df = pd.concat([usdjpy, cadjpy, audjpy, eurjpy, zarjpy , sgdjpy ], axis=1)
display(combined_df.head())

,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency,Times,Final Price,Final Price Normalized,1h_Return,...,Final Price Normalized,1h_Return,Label,Currency,Times,Final Price,Final Price Normalized,1h_Return,Label,Currency
0,2022-01-17 18:10:00,114.597,0.264026,-0.000044,0,0,2022-01-17 18:10:00,91.550,0.474199,0.000033,...,0.499865,-0.001442,0,4,2022-01-17 18:10:00,84.945,0.301345,-0.000165,0,5
1,2022-01-17 18:11:00,114.603,0.264145,0.000026,1,0,2022-01-17 18:11:00,91.557,0.474387,0.000317,...,0.499596,-0.001484,0,4,2022-01-17 18:11:00,84.953,0.301549,0.000436,1,5
2,2022-01-17 18:12:00,114.604,0.264164,0.000009,1,0,2022-01-17 18:12:00,91.555,0.474334,0.000186,...,0.499327,-0.001527,0,4,2022-01-17 18:12:00,84.954,0.301575,0.000224,1,5
3,2022-01-17 18:13:00,114.611,0.264302,0.000044,1,0,2022-01-17 18:13:00,91.556,0.474360,0.000284,...,0.499327,-0.001435,0,4,2022-01-17 18:13:00,84.959,0.301703,0.000247,1,5
4,2022-01-17 18:14:00,114.610,0.264283,0.000026,1,0,2022-01-17 18:14:00,91.549,0.474173,0.000197,...,0.502825,0.000407,1,4,2022-01-17 18:14:00,84.956,0.301626,0.000188,1,5


In [4]:
# Function to compute technical indicator
'''
Calculates each technical indicator using the entire hour for each node.
'''

# Calculate Bollinger Bands for the entire hour
def calculate_bollinger_bands(data):
    window = len(data)
    rolling_mean = data.rolling(window=window).mean()
    upper_band = rolling_mean + 2 * data.rolling(window=window).std()
    lower_band = rolling_mean - 2 * data.rolling(window=window).std()
    
    return upper_band.iloc[-1], lower_band.iloc[-1], rolling_mean.iloc[-1]

# Calculate RSI for the entire hour
def calculate_rsi(data):
    diff = data.diff(1).dropna()
    gain = diff.where(diff > 0, 0)
    loss = -diff.where(diff < 0, 0)

    avg_gain = gain.mean()
    avg_loss = loss.mean()

    if avg_loss == 0:
        rs = np.inf  # Set to infinity to avoid division by zero
    else:
        rs = avg_gain / avg_loss

    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate RCI for the entire hour
def calculate_rci(data):
    rci = data.pct_change().sum()
    return rci

# Calculate Momentum for the entire hour
def calculate_momentum(data, n=59):
    return data.diff(n).iloc[-1]

In [5]:
# Function to create a single graph (snapshot)

'''
In each graph we use data within 18 hours.
Each hour each currency pair forms a node, so each snapshot contains 18*9　nodes.
Weighted edges are generated from computing DTW of two nodes (two 1 hour series) using "Final Price Normalized"
Node features consist of "Final Price Normalized", "1h_Return", "Currency"　and technical indicator
'''

def create_graph(snapshot, node_window=60, node_stride=60):
    
    features = snapshot[[ 'Final Price Normalized','1h_Return', 'Currency', 'Final Price']].values
    # Label = snapshot['Label'].values
    
    series = []
    currencies = []
    prices = []
    h_returns = []
    Final_prices = []
    volatilities = []
    
    upper_bands = []
    lower_bands = []
    rolling_means = []
    rsis = []
    rcis = []
    momenta = []  
    
     
    for i in range(0, len(snapshot) - node_window + 1, node_stride):
        for j in range (6):
            serie = features[i:i+node_window,j]
            series.append(serie) 
            

            currency = features[i+node_window-1,j+4]
            currencies.append(currency)

            price = features[i+node_window-1, j]
            prices.append(price)
            
            Final_price = features[i:i+node_window, j+6]
            Final_prices.append(Final_price)

            h_return = features[i+node_window-1, j+2]
            h_returns.append(h_return)
            
            # Create a DataFrame with the stock price data
            df = pd.DataFrame({'Close': Final_price})

            # Calculate Bollinger Bands
            upper_band, lower_band, mean = calculate_bollinger_bands(df['Close'])
            upper_bands.append(upper_band)
            lower_bands.append(lower_band)
            rolling_means.append(mean)

            # Calculate RSI for the entire hour using the function
            rsi = calculate_rsi(df['Close'])
            rsis.append(rsi)

            # Calculate RCI for the entire hour using the function
            rci = calculate_rci(df['Close'])
            rcis.append(rci)
            
            # Calculate Momentum for the entire hour using the function
            momentum = calculate_momentum(df['Close'])
            momenta.append(momentum)
            
            all_return = features[i:i+node_window, j+2]
            
            # Create a DataFrame with the returns
            df_r = pd.DataFrame({'Return': all_return})
            # Compute volatility (standard deviation of returns)
            volatility = df_r['Return'].std()
            volatilities.append(volatility)
            
    # Edge generation 
    adjacency_matrix = np.zeros((len(series), len(series)))
    
    for i in range(len(series)):
        for j in range(i+1, len(series)):
            dtw_distance = dtw(series[i], series[j])

            # Update the maximum observed DTW distance dynamically
            max_dtw_distance = max(max_dtw_distance, dtw_distance) if 'max_dtw_distance' in locals() else dtw_distance
    
    for i in range(len(series)):
        for j in range(i+1, len(series)):
            dtw_distance = dtw(series[i], series[j])
            similarity = 1 - (dtw_distance / max_dtw_distance)

            
            adjacency_matrix[i, j] = similarity
            adjacency_matrix[j, i] = similarity
            
    np.fill_diagonal(adjacency_matrix, 1)
    # print(adjacency_matrix.shape)

    feature_matrix = np.transpose(np.vstack((prices, h_returns, currencies, upper_bands, lower_bands, rolling_means, rsis, rcis, momenta, volatilities)))

    return adjacency_matrix, feature_matrix


In [6]:
# Function to create graphs
'''
Build a graph using 18 hours of data, then move forward by 1 hour and build the next graph using the next 18 hours of data,
store them in a list graphs.
Time window of each graph is 18*60, stride is 60.

The label of each graph is the 'Lable' of the next hour's data, ("Label" is 1 if "1h_return" is larger than 1, 0 otherwise)
Because we want to use each graph(18 hours data) to predict the next hour data.
The label of each graph is a sequence of 9 binary values, each represents a currency.
'''

def create_graphs(data, time_window=18*60, stride=60):
    adjacency_matrice = []
    feature_matrice = []
    graphs = []
    labels = []
    
    # Get label data
    Label = data['Label'].values
    
    # Iterate through data with a sliding window
    for i in tqdm(range(0, len(data) - 2*time_window + 1, stride)):
    # Get a snapshot of the time series data
        snapshot = data.iloc[i:i + time_window]
        
        # Generate adjacency matrix and feature matrix from the snapshot
        adjacency_matrix, feature_matrix = create_graph(snapshot)
        
        # Get the label
        label = labels_data[i + time_window + 61]
        
        # Create edge index indicating positions of non-zero elements
        edge_index = torch.tensor([[i, j] for i in range(adjacency_matrix.shape[0]) for j in range(adjacency_matrix.shape[1]) if adjacency_matrix[i, j] != 0], dtype=torch.long).t().contiguous()
        
        # Convert to PyTorch Tensors
        x_tensor = torch.FloatTensor(feature_matrix)
        y_tensor = torch.LongTensor(label)       

        # Create a PyTorch Geometric Data object
        graph = Data(x=x_tensor, edge_index=edge_index)
        
        # Append to the lists
        graphs.append(graph)
        labels.append(y_tensor)

    return graphs, labels

In [7]:
#GCN
'''
The task is graph classification.
Output is the label of each graph which is a sequence of 9 binary values, each represents a currency.
'''

class GCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        #self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.linear1 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch

        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = self.conv2(x, edge_index, edge_weight)
        
        # Readout layer, take feature-wise average values of all node embeddings and use it as a graph feature of an input graph
        x = global_mean_pool(x, batch)
        
        # Transform a graph feature by a linear transformation
        x= self.linear1(x)
        
        #Employ a sigmoid function as an activation function of the output layer
        x = torch.sigmoid(x)
        return x

In [8]:
# Generate all snapshots. 
graphs, labels = create_graphs(combined_df)

  0%|          | 0/33947 [00:00<?, ?it/s]

In [10]:
# Split data into training and testing sets
split_ratio = 0.85
split_idx = int(split_ratio * len(graphs))

train_data = graphs[:split_idx]
train_labels = labels[:split_idx]
test_data = graphs[split_idx:]
test_labels = labels[split_idx:]

In [11]:
# Set batch size for training
batch_size = 32

# Learning rate for optimization
lr = 0.00067

# Create DataLoader for training and test data with specified batch size and shuffle
train_loader = DataLoader(list(zip(train_data, train_labels)), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(list(zip(test_data, test_labels)), batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNModel(input_dim=10, hidden_dim=128, output_dim=6).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

In [12]:
# Training loop 
num_epochs = 1200
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        graphs, labels = batch
        graphs = graphs.to(device)
        labels = labels.to(device)
        labels = labels.float()
        
        optimizer.zero_grad()
        output = model(graphs)
        output = output.float()
        
        # print(output.shape)
        # print(labels.shape)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f'Epoch:{epoch + 1} \t loss: {loss:.6f}')
        torch.save(model.state_dict(), 'GCN_2pairs_ti.dat')


Epoch:50 	 loss: 0.486165
Epoch:100 	 loss: 0.534270
Epoch:150 	 loss: 0.557625
Epoch:200 	 loss: 0.474303
Epoch:250 	 loss: 0.436851
Epoch:300 	 loss: 0.417037
Epoch:350 	 loss: 0.536666
Epoch:400 	 loss: 0.458636
Epoch:450 	 loss: 0.636786
Epoch:500 	 loss: 0.571695
Epoch:550 	 loss: 0.660089
Epoch:600 	 loss: 0.634911
Epoch:650 	 loss: 0.526739
Epoch:700 	 loss: 0.422298
Epoch:750 	 loss: 0.500811
Epoch:800 	 loss: 0.509014
Epoch:850 	 loss: 0.516645
Epoch:900 	 loss: 0.474252
Epoch:950 	 loss: 0.658246
Epoch:1000 	 loss: 0.631398
Epoch:1050 	 loss: 0.471603
Epoch:1100 	 loss: 0.588867
Epoch:1150 	 loss: 0.584940
Epoch:1200 	 loss: 0.768320


In [13]:
all_predictions = []
all_labels = []

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation during evaluation
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluating', leave=False):
        graphs, labels = batch
        graphs = graphs.to(device)
        labels = labels.to(device)
        labels = labels.float()

        # Forward pass
        output = model(graphs)
        output = output.float()

        # Convert probabilities and labels to numpy arrays for scikit-learn metrics
        output_np = output.cpu().numpy()
        labels_np = labels.cpu().numpy()

        all_predictions.append(output_np)
        all_labels.append(labels_np)

# Concatenate predictions and labels across batches
all_predictions = np.concatenate(all_predictions)
all_labels = np.concatenate(all_labels)
# Binary classification thresholding (you can adjust the threshold if needed)
threshold = 0.5
binary_predictions = (all_predictions > threshold).astype(int)

# # # Calculate evaluation metrics
precision = precision_score(all_labels, binary_predictions, average='macro')
recall = recall_score(all_labels, binary_predictions, average='macro')
roc_auc = roc_auc_score(all_labels, all_predictions)
aupr = average_precision_score(all_labels, all_predictions)
accuracy = accuracy_score(all_labels, binary_predictions)
f1 = f1_score(all_labels, binary_predictions, average='macro')

# Print or use the metrics as needed
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'AUC: {roc_auc:.4f}')
print(f'AUPR: {aupr:.4f}')
print(f'F1 Score: {f1:.4f}') 

Evaluating:   0%|          | 0/213 [00:00<?, ?it/s]

Accuracy: 0.6692
Precision: 0.6765
Recall: 0.6685
AUC: 0.7607
AUPR: 0.7738
F1 Score: 0.6652
